In [1]:
from os.path import expanduser
home = expanduser('~')
import numpy as np
import pandas as pd
from pymatgen.io.vasp.outputs import Eigenval, Outcar
import matplotlib.pyplot as plt

In [8]:
### Settings
prop_dict = {}
indep_var_name = 'U'
indep_vars = [0, 1, 2, 3, 4]
natoms_list = [32,32,16]

iv_labels = ['$P4/nmm$', '$Pba2$', '$P2/c$']
symmetries = ['P4nmm', 'Pba2', 'sm-P2c']

LOC = home + '/Projects/BaCoS2/correct_mag/is3-u-estruc'

LABEL_FONT = 14

### Functions
def read_output(prefix, loc):

    ### Reading data
    for sym, natoms in zip(symmetries, natoms_list):

        band_gaps = []
        mag_moms = []
        energies = []
        volumes = []
        des = []

        for var in indep_vars:

                suffix = f'{prefix}_{sym}-U{var}'

                ### Parsing output files
                out = Outcar(f'{loc}/OUTCAR-{suffix}')
                eig = Eigenval(f'{loc}/EIGENVAL-dos-{suffix}', occu_tol=0.02)

                ### Energy
                out.read_pattern({"energy": "energy\(sigma->0\)\s+=\s+([\d\-\.]+)"}, 
                                 reverse=True, terminate_on_match=True)
                n_atoms = len(out.charge)
                energies.append(float(out.data['energy'][0][0])/n_atoms)
                
                ### Band gap
                band_gap = np.round(eig.eigenvalue_band_properties[0],3)
                # round down band gap due to ISMEAR=-5 negative occupancy errors
                if band_gap < 0.05:
                    band_gap = 0
                band_gaps.append(band_gap)
                #  print(f'\nBand gap = {np.round(eig.eigenvalue_band_properties[0],3)}')

                ### Magnetic details
                #  unique, counts = np.unique(np.round([mag["tot"] for mag in out.magnetization[8:16]], 2), return_counts=True); print(f'{np.asarray((unique,counts)).T}') 
                co_indices = (int(n_atoms/4), int(n_atoms/2))
                mag_mom = np.mean(np.abs([np.round(mag["tot"],3) for mag in out.magnetization[co_indices[0]:co_indices[1]]])) # Average moment
                mag_moms.append(mag_mom)

                ### Volume
                out.read_pattern({"volume": "volume of cell :\s+(\d+\.\d+)*"}, 
                                 reverse=True, terminate_on_match=True)
                volumes.append(float(out.data['volume'][0][0]))

        prop_dict.update({(sym, 'indep_var') : indep_vars})
        prop_dict.update({(sym, 'e_per_atom') : energies})
        prop_dict.update({(sym, 'band_gap') : band_gaps})
        prop_dict.update({(sym, 'mag_mom') : mag_moms})
        prop_dict.update({(sym, 'volume') : volumes})
        prop_dict.update({(sym, 'v_per_atom') : [v/natoms for v, natoms in zip(volumes, natoms_list)]})
        prop_dict.update({(sym, 'de_per_atom') : 1000*(np.array(energies)-np.array(prop_dict[symmetries[0],'e_per_atom']))})
            
    df = pd.DataFrame(prop_dict)
#     print(df[symmetries[0]]['e_per_atom'])
    
    ### Energy relative to P4/nmm
#     for sym in symmetries:
#     df['de_per_atom'] = [df[sym]['e_per_atom'] - df[symmetries[0]]['e_per_atom'] for sym in symmetries]
    
    return df


def plot_var(prefix, df, dep_var, axis_label, labels, xlims=[None,None], ylims=[None,None]):
    
    fig = plt.figure(figsize=(5.5,3.4))
    
    x = df[symmetries[0]].indep_var
    
    markers = ['o', '^', 's']
    
    for sym, label, marker in zip(symmetries, labels, markers):
        y = df[sym][dep_var]
        plt.plot(x, y, marker=marker, label=label, clip_on=False)
    
    # Plot customization
#     plt.axvline(x=13, color='k', linestyle='--')
    plt.gca().xaxis.label.set_fontsize(LABEL_FONT)
    plt.gca().yaxis.label.set_fontsize(LABEL_FONT)
    plt.gca().tick_params(labelsize=LABEL_FONT)
    plt.legend()
    plt.grid(True)
    plt.xlabel('$U_{eff}$ (eV)'); plt.ylabel(axis_label)

#     plt.legend(fancybox=True, framealpha=0.5, fontsize=LABEL_FONT-2, ncol=3) 
    plt.legend(loc=(0,1.01), ncol=3, fontsize=LABEL_FONT-2) 

    # dep_var-dependent customization        
    if dep_var == 'volume':
        plt.axhline(y=748.2, color='k', linestyle=':')
        
    if dep_var == 'v_per_atom':
        plt.axhline(y=748.2/32, color='k', linestyle=':')
        
    if dep_var == 'band_gap':
        plt.axhline(y=0.23, color='k', linestyle=':') 
        
    plt.xlim(xlims); plt.ylim(ylims)
    plt.show()

    # Saving plot
    fig.savefig(f'{prefix}_{dep_var}.png', bbox_inches='tight', dpi=400)
    
# def plot_eqn_of_state(df):
#     fig = plt.figure(figsize=(4.42,3))
    
#     for sym in symmetries:
#         x = df[sym]['volume']
#         y = df[sym]['e_per_atom']
#         plt.plot(x, y, 'o-', label=sym, clip_on=False)
        
#         # Plot customization
#         plt.gca().xaxis.label.set_fontsize(LABEL_FONT)
#         plt.gca().yaxis.label.set_fontsize(LABEL_FONT)
#         plt.gca().tick_params(labelsize=LABEL_FONT)
#         plt.legend()
#         plt.grid(True)
#         plt.xlabel('Volume ($\AA^3$)'); plt.ylabel('Energy (eV/atom)')
#         plt.legend(loc=(0,1.01), ncol=3, fontsize=LABEL_FONT-2) 

In [9]:
### Variables to plot
variables = ['de_per_atom', 'band_gap', 'v_per_atom', 'mag_mom']

axis_label_dict = {'de_per_atom' : '$E-E(P4/nmm)$ (meV/atom)', 'e_per_atom' : 'Energy (eV/atom)', 
                   'band_gap' : 'Band Gap (eV)', 'mag_mom' : 'Avg. Co  $|\mu_B|$', 
                   'v_per_atom' : 'Volume/atom ($\AA^3$)'}

ylim_dict = {'de_per_atom' : [None, None], 
             'band_gap' : [None, None], 
             'mag_mom' : [None, None], 
             'v_per_atom' : [None, None]}

# df = read_output('struc', LOC)
df = read_output('struc', LOC)

for var in variables:
    plot_var('struc', df, var, axis_label_dict[var], iv_labels, ylims=ylim_dict[var]) 

ValueError: arrays must all be same length

In [10]:
prop_dict

{('P4nmm', 'indep_var'): [0, 1, 2, 3, 4],
 ('P4nmm', 'e_per_atom'): [-5.439633660625,
  -5.258772435625,
  -5.10069713125,
  -4.949399896875,
  -4.937996464375],
 ('P4nmm', 'band_gap'): [0, 0, 0, 0.063, 1.023],
 ('P4nmm', 'mag_mom'): [1.399,
  1.7229999999999999,
  1.902,
  2.0934999999999997,
  2.515],
 ('P4nmm', 'volume'): [706.11, 721.64, 730.68, 767.29, 775.5],
 ('P4nmm', 'v_per_atom'): [22.0659375, 22.55125, 45.6675],
 ('P4nmm', 'de_per_atom'): array([0., 0., 0., 0., 0.]),
 ('Pba2', 'indep_var'): [0, 1, 2, 3, 4],
 ('Pba2', 'e_per_atom'): [-5.4396342175,
  -5.271162431875,
  -5.138792561875,
  -5.022193966875,
  -4.9254918778125],
 ('Pba2', 'band_gap'): [0, 0.254, 0.703, 0.88, 1.176],
 ('Pba2', 'mag_mom'): [1.3995,
  1.9860000000000002,
  2.22125,
  2.3594999999999997,
  2.4610000000000003],
 ('Pba2', 'volume'): [706.09, 741.16, 758.55, 769.7, 779.47],
 ('Pba2', 'v_per_atom'): [22.0653125, 23.16125, 47.409375],
 ('Pba2',
  'de_per_atom'): array([-5.56875000e-04, -1.23899963e+01, -3